In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import platform

Importing plotly failed. Interactive plots will not work.


In [38]:
# 1️⃣ CSV 로드 + 기본 필터링
df = pd.read_csv("data/1_2_(로우데이터_합본.csv)목적별_국적별_입국(05년1월~25년5월).csv", encoding='cp949')

df = df[
    (~df['국적'].str.endswith('주', na=False)) &
    (~df['국적'].str.endswith('기타', na=False)) &
    (~df['국적'].isin(['소 계', '전 체', '미상', '기 타', '교포', '중 동'])) &
    (~df['목적'].isin(['소 계', '전 체']))
]

df = df[df['국적'].notna()]

# 2️⃣ 컬럼 나누기
fixed_cols = ['국적', '목적']
date_cols = [col for col in df.columns if ('년' in col and '월' in col)]

# ✅ 3️⃣ 제외 범위: 2020년 3월 ~ 2022년 6월
cols_to_drop = []
for col in date_cols:
    year = int(col.split('년')[0])
    month = int(col.split('년')[1].replace('월', '').zfill(2))
    ym = year * 100 + month
    if 202003 <= ym <= 202206:
        cols_to_drop.append(col)

# ✅ 마지막 2개 열 제외하고 범위 제외까지
date_cols_final = [col for col in date_cols if col not in cols_to_drop][:-2]
cols_to_use = fixed_cols + date_cols_final

df = df[cols_to_use]

# 4️⃣ 날짜 컬럼 쉼표 제거 + 소수점 버리기
for col in date_cols_final:
    df[col] = df[col].astype(str).str.replace(',', '').astype(float).astype(int)

# 5️⃣ 헤더 행 추가
new_row = pd.DataFrame(
    [[None, None] + ['인원(명)'] * len(date_cols_final)],
    columns=cols_to_use
)
df_final = pd.concat([new_row, df], ignore_index=True)

# 6️⃣ 저장
df_final.to_csv("data/목적별_국적별_입국_소계제거.csv", index=False, encoding='cp949')
